In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
data = pd.read_csv("../input/The Movies Dataset/movies_metadata.csv", low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


여기서 코사인 유사도에 사용할 데이터는 title과 overview 열이다. 좋아하는 영화를 입력하면 줄거리가 유사한 영화를 찾아 추천하는 시스템을 만들 것이다.

In [5]:
data = data.head(20000)

In [6]:
print(f"overview 열의 결측값 수: {data['overview'].isnull().sum()}")

overview 열의 결측값 수: 135


여기서는 Null을 그냥 빈 값으로 대체하겠다.

In [7]:
data["overview"] = data["overview"].fillna("")

overview 열에 대해 TF-IDF 행렬을 구한 후 행렬의 크기를 출력해보자.

In [8]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(data["overview"])
tfidf_matrix.shape

(20000, 47487)

20000 개의 문서에서 47487개의 단어가 사용되었음을 알 수 있다. 20000개의 문서 벡터에 대해 상호 간 코사인 유사도를 구해보자.

In [9]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(20000, 20000)

In [10]:
title_to_index = dict(zip(data["title"], data.index))
title_to_index["Father of the Bride Part II"]

4

제목을 입력하면 코사인 유사도를 통해 overview가 가장 유사한 10개의 영화를 찾아내는 함수를 만들어보자.

In [11]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = title_to_index[title]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    movie_indices = [idx[0] for idx in sim_scores]
    return data["title"].iloc[movie_indices]

In [16]:
get_recommendations("The Dark Knight Rises")

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object